## Solve Model balancing problem with kinetic parameters as variables

In [1]:
import numpy as np
from equilibrator_api import Q_, R, default_T
from model_balancing import _ln_kcatr, _ln_conc_enz, _driving_forces, solve
import cvxpy as cp

In [2]:
use_regulation = False
args = {}

# choose solver

if cp.MOSEK in cp.installed_solvers():
    args["solver"] = cp.MOSEK
else:
    #args["solver"] = cp.SCS
    args["solver"] = cp.ECOS

In [3]:
# Network topology

args["S"] = np.matrix(
    [
        [-1, 0, 0],
        [1, -1, -1],
        [0, 1, 1]
    ], dtype=float
)

if use_regulation:
    args["A_act"] = np.matrix(
        [
            [0, 1, 2],
            [0, 0, 0],
            [0, 0, 0]
        ], dtype=float
    )
    args["A_inh"] = np.matrix(
        [
            [0, 0, 0],
            [0, 0, 0],
            [2, 0, 0]
        ], dtype=float
    )
else:
    args["A_act"] = np.matrix(
        [
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0]
        ], dtype=float
    )
    args["A_inh"] = np.matrix(
        [
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0]
        ], dtype=float
    )

In [4]:
# Priors for Kinetic and Thermodynamic parameters

args["Keq_gmean"] = Q_([57, 0.13, 3200], "")  # geometric mean (assuming a standard concentration of 1M)
args["Keq_ln_cov"] = np.array(
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
    ]
)  # log-scale covariance

args["kcatf_gmean"] = Q_([20.0, 8.0, 10.0], "1/s")  # geometric mean
args["kcatf_ln_cov"] = np.array(
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
    ]
) # log-scale covariance

args["kcatr_gmean"] = Q_([3.5e-3, 6.2e4, 3.1e-1], "1/s")  # geometric mean
args["kcatr_ln_cov"] = np.array(
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
    ]
) # log-scale covariance

# the K parameters are only for existing connections in the S, A_act and A_inh matrices.
# the order of values is "metabolite-first".
args["Km_gmean"] = Q_([1e-2, 1e-4, 1e-4, 1e-3, 1e-1, 1e-1], "M")
args["Km_ln_cov"] = np.array(
    [
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1],
    ]
) # log-scale covariance

args["Ka_gmean"] = Q_([1, 1], "M")
args["Ka_ln_cov"] = np.array(
    [
        [1, 0],
        [0, 1],
    ]
)

args["Ki_gmean"] = Q_([1], "M")
args["Ki_ln_cov"] = np.array(
    [
        [1],
    ]
)

In [5]:
# condition dependent data (columns represent conditions)

args["fluxes"] = Q_(
    [
        [2.0, 1.5, 2.5, 2.5],
        [1.0, 1.0, 2.0, 0.5],
        [1.0, 0.5, 0.5, 2.0],
    ], "mM/s"
)

args["conc_enz_gmean"] = Q_(
    [
        [1e-3, 1e-3, 1e-3, 3e-3],
        [2e-3, 3e-3, 1e-3, 2e-3],
        [1e-3, 2e-3, 2e-3, 1e-3],
    ], "M")  # geometric mean

args["conc_enz_gstd"] = np.array(
    [
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
    ]
)  # geometric standard deviation

args["conc_met_gmean"] = Q_(
    [
        [2e-3, 1e-2, 2e-3, 1e-3],
        [2e-4, 1e-4, 3e-4, 1e-4],
        [1e-5, 1e-5, 1e-5, 1e-5],
    ], "M"
)  # geometric mean

args["conc_met_gstd"] = np.array(
    [
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
    ]
)  # geometric standard deviation

In [6]:
fluxes, S, A_act, A_inh = args["fluxes"], args["S"], args["A_act"], args["A_inh"]
ln_Keq = np.log(args["Keq_gmean"].m_as(""))
ln_kcatf = np.log(args["kcatf_gmean"].m_as("1/s"))
ln_Km = np.log(args["Km_gmean"].m_as("M"))
ln_Ka = np.log(args["Ka_gmean"].m_as("M"))
ln_Ki = np.log(args["Ki_gmean"].m_as("M"))
ln_conc_met = np.log(args["conc_met_gmean"].m_as("M"))

args["kcatr_gmean"] = np.exp(_ln_kcatr(ln_kcatf, ln_Km, ln_Keq, S).value) * Q_("1/s")

In [7]:
# check that the driving forces at the mode are all positive
if (_driving_forces(ln_Keq, ln_conc_met, S).value >= 1e-6).all():
    print("Feasible")
else:
    raise Exception("Infeasible")

Feasible


In [8]:
for rate_law in ["CM"]:#["S", "1S", "SP", "1SP", "CM"]:
    print(f"************ rate law = {rate_law} **************\n")
    args["conc_enz_gmean"] = np.exp(
        _ln_conc_enz(
            ln_Keq, ln_kcatf, ln_Km, ln_Ka, ln_Ki, ln_conc_met,
            fluxes, S, A_act, A_inh , rate_law=rate_law).value
    ) * Q_("M")
    args["rate_law"] = rate_law

    solve(
        **args,
    )

************ rate law = CM **************

initial Z-score = 0.00e+00
optimal Z-score = 4.69e-09

Metabolite concentrations (M) =
 [[2.00000018e-03 1.00000007e-02 2.00000018e-03 1.00000009e-03]
 [2.00000018e-04 1.00000015e-04 3.00000019e-04 1.00000014e-04]
 [9.99999951e-06 9.99999908e-06 9.99999966e-06 9.99999908e-06]]

Enzyme concentrations (M) =
 [[0.00160275 0.00022503 0.00263183 0.00262951]
 [0.00030468 0.00108325 0.00044827 0.00054162]
 [0.00060004 0.00055004 0.00021668 0.00220018]]

Driving forces (RT) =
 [[ 6.34565871  8.64824373  5.94019363  6.34565866]
 [ 0.95554139  0.26239431  1.36100645  0.2623943 ]
 [11.06665445 10.37350737 11.47211952 10.37350737]]

η(thr) =
 [[1.   1.   1.   1.  ]
 [0.62 0.23 0.74 0.23]
 [1.   1.   1.   1.  ]]

η(kin) =
 [[0.06 0.33 0.05 0.05]
 [0.67 0.5  0.75 0.5 ]
 [0.17 0.09 0.23 0.09]]

η(reg) =
 [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]




All Z-scores
--------------------------------------------------
enzymes =  0.0
metabolites =  0.0
Keq =  0